In [19]:
# 분야 2 | 유선 네트워크 장비의 신속한 조치를 위한 경보 유형 분류

# 아래의 코드는 다음과 같은 순서로 구성되어 있습니다.
# 1. 데이터 로드 및 전처리
# 2. 모델 학습 및 예측
# 3. 결과 제출
# 본 코드는 결과물 제출까지의 이해를 돕기위한 코드로, 반드시 이 구조를 따를 필요는 없습니다.
# 데이터 전처리, 모델의 구성 등에서 다양한 시도를 하시되, 올바른 채점을 위한 최종 결과물의 형태에 유의하시기 바랍니다.

#### 버전: Python 3.8.15

In [20]:
# 1. 데이터 로드
import numpy as np
import pandas as pd

def ppr_data(q2_train, q2_test, alarmmsg):
    # TODO: 데이터 전처리 코드 구현 ---------- #
    # slotna 변수 생성
    q2_train['slotna'] = q2_train['slot'].isna().astype(int)
    q2_test['slotna'] = q2_test['slot'].isna().astype(int)

    # alarmmsg 전처리
    q2_train = pd.merge(q2_train, alarmmsg, how='left')
    q2_test = pd.merge(q2_test, alarmmsg, how='left')

    q2_train.drop(['alarmmsg_original'], axis=1, inplace=True)
    q2_train.rename(columns={"alarmmsg_edited": "alarmmsg_original"}, inplace=True)

    q2_test.drop(['alarmmsg_original'], axis=1, inplace=True)
    q2_test.rename(columns={"alarmmsg_edited": "alarmmsg_original"}, inplace=True)
    
    # alarmno 자릿수
    for i in range(len(q2_train)):
        q2_train['alarmno'][i] = int(str(q2_train['alarmno'][i])[0:6])
    for i in range(len(q2_test)):
        q2_test['alarmno'][i] = int(str(q2_test['alarmno'][i])[0:6])
    
    # 필요 없는 변수 drop
    q2_train.drop(labels = ['slot', 'port', 'sva', 'root_cause_domain'], axis = 1, inplace = True)
    q2_test.drop(labels = ['slot', 'port', 'sva', 'root_cause_domain'], axis = 1, inplace = True)

    # 열 순서 정렬
    q2_train = q2_train[[col for col in q2_train.columns if col != 'root_cause_type'] + ['root_cause_type']]

    # time 변수 datetime으로 타입 변경
    q2_train['alarmtime'] = pd.to_datetime(q2_train['alarmtime'])
    q2_test['alarmtime'] = pd.to_datetime(q2_test['alarmtime'])

    # train, test 데이터 합치기
    q2_train['train'] = 1
    q2_test['train'] = 0
    q2_train = q2_train[[col for col in q2_train.columns if col != 'root_cause_type'] + ['root_cause_type']]
    data_total = pd.concat([q2_train, q2_test])

    # alarmtime 변수로 지속시간 변수 추가 및 time변수 삭제
    data = data_total.groupby('ticketno')['alarmtime'].agg(**{'min_time':'min', 'max_time':'max'}).reset_index()
    data['duration'] = data['max_time'] - data['min_time']
    for i in range(len(data['duration'])):
      data['duration'][i] = data['duration'][i].seconds / 60

    # 전표별 level 당 개수 변수 생성
    level = data_total.groupby(['ticketno','alarmlevel']).size().unstack(fill_value = 0).reset_index()
    data['level3'] = level[3]
    data['level4'] = level[4]
    data['level5'] = level[5]
    data['level7'] = level[7]
    data.drop(labels=['max_time'], axis = 1, inplace = True)

    # 전표별 alarmmsg 이어붙이기
    msg = data_total.groupby("ticketno")["alarmmsg_original"].apply(list).reset_index()

    # 전표별 데이터 합치기
    data = pd.merge(data, msg, how='left')
    data = pd.merge(data, data_total.groupby('ticketno')['alarmno'].agg(**{'alarmno':max}).reset_index())
    data = pd.merge(data, data_total.groupby('ticketno')['train'].agg(**{'train':max}).reset_index())
    data = pd.merge(data, data_total.groupby('ticketno')['slotna'].agg(**{'slotna': max}).reset_index())
    data = pd.merge(data, data_total.groupby('ticketno')['root_cause_type'].agg(**{'type':max}).reset_index())   

    data['alarmmsg_original'] = data['alarmmsg_original'].apply(lambda msg_list: ' '.join(msg_list))
    
    # data 시간별로 sort
    data.sort_values('min_time', inplace = True)

    # train, test 분리
    data_train = data[data['train'] == 1]
    data_test = data[data['train'] == 0]
    data_train.drop(labels = ['train'], axis = 1, inplace = True)
    data_test.drop(labels = ['train'], axis = 1, inplace = True)

    # x, y 변수 분리
    x_train_df = data_train.iloc[:, :-1]
    y_train_df = data_train['type']
    x_test_df = data_test.iloc[:, :-1]

    # ------------------------------------- #
    return x_train_df, y_train_df, x_test_df


train_df = pd.read_csv("C:/Users/장희중학부재학통계학과/OneDrive - 고려대학교/바탕 화면/hj/공모전/통신망 해커톤/data/Q2_data/Q2_train.csv")
test_df = pd.read_csv("C:/Users/장희중학부재학통계학과/OneDrive - 고려대학교/바탕 화면/hj/공모전/통신망 해커톤/data/Q2_data/Q2_test.csv")
alarmmsg_df = pd.read_excel("C:/Users/장희중학부재학통계학과/OneDrive - 고려대학교/바탕 화면/hj/공모전/통신망 해커톤/data/Q2_data/alarmmsg_edit.xlsx")


x_train_df, y_train_df, x_test_df = ppr_data(train_df, test_df, alarmmsg_df)

C:\Users\장희중학부재학통계학과\AppData\Local\Temp\ipykernel_10264\3149815476.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  q2_train['alarmno'][i] = int(str(q2_train['alarmno'][i])[0:6])
C:\Users\장희중학부재학통계학과\AppData\Local\Temp\ipykernel_10264\3149815476.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  q2_test['alarmno'][i] = int(str(q2_test['alarmno'][i])[0:6])
C:\Users\장희중학부재학통계학과\AppData\Local\Temp\ipykernel_10264\3149815476.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/in

In [21]:
# 2. 모델 학습 및 예측
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import ComplementNB
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report

class MyModel:
    def __init__(self) -> None:
        self.model = None

    def train(self, x_train, y_train):
        # TODO: 모델 학습 코드 구현 ---------- #
        
        
        # TfidfVertorizer로 메시지 수치화 후 Multinomial naive bayes 모델링

        # Create transformers
        message_vectorizer = TfidfVectorizer(analyzer = 'char', sublinear_tf = True, max_df=0.8)
        additional_scaler = MinMaxScaler()

        # Define column indices for the ColumnTransformer
        message_col_idx = 'alarmmsg_original'
        additional_cols = ['alarmno', 'duration', 'level3', 'level4', 'level5', 'level7', 'slotna']

        # Create the ColumnTransformer
        preprocessor = ColumnTransformer(
            transformers=[
                ('message', message_vectorizer, message_col_idx),
                ('additional', additional_scaler, additional_cols)
            ],
            remainder='passthrough'
        )

        
        # Create individual classifiers
        classifier1 = ComplementNB(alpha=11)
        classifier2 = CatBoostClassifier(n_estimators=80, random_seed = 425)  # Adjust hyperparameters


        # Create the VotingClassifier
        voting_classifier = VotingClassifier(
            estimators=[
                ('nb', classifier1),
                ('cb', classifier2)
            ],
            voting='soft',  # You can use 'hard' or 'soft' voting
            
        )
        

        # Create a pipeline with the ColumnTransformer and the classifier
        pipeline = Pipeline([
            ('preprocessor', preprocessor),
            ('classifier', voting_classifier)
        ])

        x_train = x_train[['alarmmsg_original'] + additional_cols]

        # Train the pipeline
        self.model = pipeline.fit(x_train, y_train)


        # --------------------------------- #
        pass

    def predict(self, x_test):
        # 1. ticketno 컬럼은 입력받은 값으로 채우고,
        # 2. pred 컬럼은 모두 'LinkCut' 값으로 채운 데이터프레임 생성
        pred_df = pd.DataFrame({'ticketno': x_test['ticketno'].values, 'root_cause_type': self.model.predict(x_test)})
        pred_df.sort_values('ticketno', inplace=True)
        pred_df.reset_index(drop=True, inplace=True)
        return pred_df

model = MyModel()
model.train(x_train_df, y_train_df)
y_pred = model.predict(x_test_df) # 0.935752

import joblib
joblib.dump(model, 'model_Q2.pkl')

Learning rate set to 0.5
0:	learn: 0.4644307	total: 6.36ms	remaining: 503ms
1:	learn: 0.2793566	total: 11ms	remaining: 429ms
2:	learn: 0.1897160	total: 17.1ms	remaining: 438ms
3:	learn: 0.1321258	total: 21.7ms	remaining: 412ms
4:	learn: 0.0965363	total: 26.3ms	remaining: 394ms
5:	learn: 0.0756511	total: 32.1ms	remaining: 396ms
6:	learn: 0.0617280	total: 37.9ms	remaining: 396ms
7:	learn: 0.0523993	total: 42.6ms	remaining: 383ms
8:	learn: 0.0444644	total: 48.3ms	remaining: 381ms
9:	learn: 0.0424417	total: 52.5ms	remaining: 367ms
10:	learn: 0.0387120	total: 58ms	remaining: 364ms
11:	learn: 0.0346009	total: 63.6ms	remaining: 361ms
12:	learn: 0.0312737	total: 68.3ms	remaining: 352ms
13:	learn: 0.0281378	total: 75.5ms	remaining: 356ms
14:	learn: 0.0260777	total: 81.6ms	remaining: 353ms
15:	learn: 0.0246032	total: 87.4ms	remaining: 350ms
16:	learn: 0.0233232	total: 93.5ms	remaining: 347ms
17:	learn: 0.0213925	total: 99.4ms	remaining: 342ms
18:	learn: 0.0191850	total: 107ms	remaining: 343ms
19

['model_Q2.pkl']

In [768]:
# 3. 결과 제출
# 본 코드는 제출되는 파일의 형태에 대한 가이드로, 반드시 아래 구조를 따를 필요 없이 자유롭게 코드를 작성해도 무방합니다.
# 제출 포맷에 대해선 data/Q2_label_sample.csv를 참조하세요.
#
# 분야 2의 경우, 전표(ticket) 하나에 하나의 근원장애(root_cause_type)을 매칭해야 합니다.
#   주의: 경보(alarm) 개수와 전표(ticket) 개수는 다르며, 예측할 대상은 전표입니다.
#   주의: ticketno 컬럼 기준으로 오름차순 정렬이 필요합니다.
# 분야 2의 제출 파일은 2개 컬럼 [ticketno, root_cause_type]을 가져야 합니다.

def submitResult(pred):
    try:
        label = pd.read_csv('C:/Users/장희중학부재학통계학과/OneDrive - 고려대학교/바탕 화면/hj/공모전/통신망 해커톤/data/Q2_data/Q2_label_sample.csv')
        # ticketno 순서와 개수가 일치하는지 확인
        if (label['ticketno'] == pred['ticketno']).all():
            print("Check: ticketno 순서와 샘플 수가 일치합니다.")
        else:
            print("Warning: 테스트 세트와 모델 예측의 ticketno가 일치하지 않습니다.")
            return

        pred.to_csv('Q2_submitResult.csv', index=False)
        print("Done : Q2_submitResult.csv 파일로 저장되었습니다.")
    except Exception as e:
        # 예외가 발생한 경우 오류 메시지 출력
        print("Error:", e)

submitResult(y_pred)

Check: ticketno 순서와 샘플 수가 일치합니다.
Done : Q2_submitResult.csv 파일로 저장되었습니다.
